In [1]:
import numpy as np
import pandas as pd
import matplotlib
import anndata as ad
import pickle
import sys

import plotly.io as pio
import plotly.express as px
import plotly.graph_objs as go
import plotly.figure_factory as ff
pd.options.plotting.backend = 'matplotlib'
pio.renderers.default = 'iframe'

# Quipi

In [2]:
# Read in cleaned version of DF for testing
master_df = pd.read_pickle("/Users/hwismer/Documents/QuIPI/data/clean/quipi_log2_tpm.pi")
master_df

,patient,sample_name,indication,compartment,archetype,sample_type,sample_type_cat,x_umap1,x_umap2,RPL5P14,...,ENSG00000215067,ARL9,PSMD7,TOMM20P1,IGLVIV.53,X.4140,RAI14.DT,TAF1B,PRRX1,COIL
0,IPIADR001,IPIADR001.T1,ADR,Myeloid,Unclassified,T1,T,NaN,NaN,0.000000,...,2.639232,0.000000,6.348728,0.000000,0.0,0.000000,0.0,3.320485,-0.043943,2.032101
1,IPIADR001,IPIADR001.T1,ADR,Stroma,Unclassified,T1,T,NaN,NaN,-0.556393,...,1.891419,-1.643856,7.226701,-0.943416,0.0,0.000000,0.0,2.066950,1.891419,3.010780
2,IPIADR001,IPIADR001.T1,ADR,T_cell,Unclassified,T1,T,NaN,NaN,0.000000,...,2.130931,0.000000,5.717265,0.000000,0.0,0.000000,0.0,2.942984,-3.643856,2.983678
3,IPIADR001,IPIADR001.T1,ADR,Tumor,Unclassified,T1,T,NaN,NaN,0.000000,...,-0.761213,1.244887,7.780376,0.000000,0.0,0.000000,0.0,2.330558,1.739848,2.792855
4,IPIADR002,IPIADR002.T1,ADR,Stroma,Unclassified,T1,T,NaN,NaN,2.010780,...,3.462052,0.000000,6.098664,0.000000,0.0,0.344828,0.0,5.179113,6.473137,-0.915936
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1515,IPISRC040,IPISRC040.T1,SRC,Treg,Unclassified,T1,T,NaN,NaN,0.000000,...,1.959770,0.000000,6.638508,0.000000,0.0,0.000000,0.0,2.784504,1.704872,2.144046
1516,IPISRC042,IPISRC042.T1,SRC,Myeloid,Unclassified,T1,T,NaN,NaN,-3.184425,...,1.925999,-1.556393,5.758623,0.000000,0.0,0.000000,0.0,1.584963,-2.473931,0.839960
1517,IPISRC042,IPISRC042.T1,SRC,Stroma,Unclassified,T1,T,NaN,NaN,0.000000,...,0.000000,-3.643856,4.069530,0.000000,0.0,0.000000,0.0,0.111031,7.626950,1.589763
1518,IPISRC042,IPISRC042.T1,SRC,T_cell,Unclassified,T1,T,NaN,NaN,0.000000,...,3.675816,0.000000,6.392661,0.000000,0.0,0.000000,0.0,2.575312,-3.321928,3.111031


In [3]:
master_df

,patient,sample_name,indication,compartment,archetype,sample_type,sample_type_cat,x_umap1,x_umap2,RPL5P14,...,ENSG00000215067,ARL9,PSMD7,TOMM20P1,IGLVIV.53,X.4140,RAI14.DT,TAF1B,PRRX1,COIL
0,IPIADR001,IPIADR001.T1,ADR,Myeloid,Unclassified,T1,T,NaN,NaN,0.000000,...,2.639232,0.000000,6.348728,0.000000,0.0,0.000000,0.0,3.320485,-0.043943,2.032101
1,IPIADR001,IPIADR001.T1,ADR,Stroma,Unclassified,T1,T,NaN,NaN,-0.556393,...,1.891419,-1.643856,7.226701,-0.943416,0.0,0.000000,0.0,2.066950,1.891419,3.010780
2,IPIADR001,IPIADR001.T1,ADR,T_cell,Unclassified,T1,T,NaN,NaN,0.000000,...,2.130931,0.000000,5.717265,0.000000,0.0,0.000000,0.0,2.942984,-3.643856,2.983678
3,IPIADR001,IPIADR001.T1,ADR,Tumor,Unclassified,T1,T,NaN,NaN,0.000000,...,-0.761213,1.244887,7.780376,0.000000,0.0,0.000000,0.0,2.330558,1.739848,2.792855
4,IPIADR002,IPIADR002.T1,ADR,Stroma,Unclassified,T1,T,NaN,NaN,2.010780,...,3.462052,0.000000,6.098664,0.000000,0.0,0.344828,0.0,5.179113,6.473137,-0.915936
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1515,IPISRC040,IPISRC040.T1,SRC,Treg,Unclassified,T1,T,NaN,NaN,0.000000,...,1.959770,0.000000,6.638508,0.000000,0.0,0.000000,0.0,2.784504,1.704872,2.144046
1516,IPISRC042,IPISRC042.T1,SRC,Myeloid,Unclassified,T1,T,NaN,NaN,-3.184425,...,1.925999,-1.556393,5.758623,0.000000,0.0,0.000000,0.0,1.584963,-2.473931,0.839960
1517,IPISRC042,IPISRC042.T1,SRC,Stroma,Unclassified,T1,T,NaN,NaN,0.000000,...,0.000000,-3.643856,4.069530,0.000000,0.0,0.000000,0.0,0.111031,7.626950,1.589763
1518,IPISRC042,IPISRC042.T1,SRC,T_cell,Unclassified,T1,T,NaN,NaN,0.000000,...,3.675816,0.000000,6.392661,0.000000,0.0,0.000000,0.0,2.575312,-3.321928,3.111031


In [ ]:
#fig = px.area(master_df, x=master_df.index, y="NCAM1", color="sample_type_cat")
#fig.show()

In [ ]:
fig = ff.create_distplot([master_df[master_df.names == a].value for a in df.names.unique()], df.names.unique(), bin_size=[.1, .25, .5, 1])
fig.show()

In [ ]:
data

In [4]:
master_df_genes = master_df[["sample_type_cat","NCAM1","DLK1"]]

master_df_T = master_df_genes[master_df_genes["sample_type_cat"] == "T"][["NCAM1"]]["NCAM1"].to_list()
master_df_N = master_df_genes[master_df_genes["sample_type_cat"] == "N"][["NCAM1"]]["NCAM1"].to_list()
#master_df_genes

data = [master_df_N, master_df_T]
group_labels = ["N","T"]

#n = [master_df_N[col] for col in master_df_N.columns]
#t = [master_df_T[col] for col in master_df_T.columns]

#data = [n,t]

fig = ff.create_distplot(data, group_labels, 
                         bin_size=.25,
                         show_hist=False,
                         show_rug=False)

#fig.add_scatter(x=data[0], y=y1,fill='tozeroy', mode='none' , fillcolor='red')


fig.update_layout(template='simple_white',width=750, height=400)
fig.show()

In [ ]:

import plotly
import plotly.figure_factory as ff
plotly.offline.init_notebook_mode()
import numpy as np

# data with different sizes
x1 = np.random.randn(300)-2  
x2 = np.random.randn(200)  
x3 = np.random.randn(4000)+2  
x4 = np.random.randn(50)+4  

# Group data together
hist_data = [x1, x2, x3, x4]

# use custom names
group_labels = ['x1', 'x2', 'x3', 'x4']

# Create distplot with custom bin_size
fig = ff.create_distplot(hist_data, group_labels, bin_size=.2)

fig.show()

# change that if you don't want to plot offline
#plotly.offline.plot(fig, filename='Distplot with Multiple Datasets')

In [ ]:
log_df[["Patient","sample_name"]]

In [ ]:
sample_type = master_df["sample_name"].str.split(".")
types = [stype[1] for stype  in sample_type]
master_df["sample_type"] = types

In [ ]:
master_df["sample_type"] = types

In [ ]:
master_df["sample_type"].str[0]

In [ ]:
master_df["sample_type_simple"] = master_df["sample_type"].str[0]
master_df

In [ ]:
non_genes = ["Patient", "sample_name", "sample_type", "Indication", "Compartment", "Archetype", "X_umap1", "X_umap2"]

In [ ]:
len(list(set(master_df.columns) - set(non_genes)))

# Create Cleaned Aggregate DF

In [ ]:
#data_path = "./data/230811_Combined_TPM_clean_Tumor_filtered_ALL_T1_T2_INDIVIDUAL_PATIENTS.tsv"
#data_path = "./data/Combined_ALL.tsv"
data_path = "./data/raw/Combined_ALL.tsv"
umap_path = "./data/raw/Pancan_10f_UMAP_COORD.csv"

In [ ]:
umap_df = pd.read_csv(umap_path, sep = ",")
umap_df = umap_df.set_index("Sample")
umap_df = umap_df.rename_axis("sample_name")
data_df = pd.read_csv(data_path, sep = "\t")

In [ ]:
new_cols = ["sample_name", "Compartment", "Patient", "Indication", "Archetype"]
rest_of_cols = list(set(data_df.columns) - set(new_cols))
sorted_df = data_df[new_cols + rest_of_cols]

multiind = sorted_df.set_index(["Patient","sample_name","Indication","Compartment","Archetype"]).sort_index()
merged_df = pd.merge(multiind,umap_df,left_index=True,right_index=True,how="outer")

tidy_df = merged_df.reset_index()

#merged_df.to_pickle("master_df.pi")

In [ ]:
log_df = master_df.copy()
cats = ["Patient","sample_name","Indication","Compartment","Archetype","sample_type","sample_type_simple","X_umap1","X_umap2"]
nums = list(set(log_df.columns) - set(cats))
nums
new_master = log_df[cats + nums].rename(columns = {"Patient":"patient",
                                      "Indication":"indication",
                                      "Compartment":"compartment",
                                      "Archetype":"archetype",
                                      "sample_type_simple":"sample_type_cat",
                                      "X_umap1":"x_umap1",
                                      "X_umap2":"x_umap2"})

In [ ]:
new_master[nums] = new_master[nums].apply(lambda x: np.log2(x.where(x > 0)).fillna(0))

In [ ]:
new_master

In [ ]:
df = pd.read_pickle("/Users/hwismer/Documents/QuIPI/data/clean/quipi_raw_tpm.pi")
df

In [ ]:
quipi_raw = pd.read_pickle("/Users/hwismer/Documents/QuIPI/data/clean/quipi_raw_tpm.pi")
quipi_log2 = pd.read_pickle("/Users/hwismer/Documents/QuIPI/data/clean/quipi_log2_tpm.pi")
quipi_log10 = pd.read_pickle("/Users/hwismer/Documents/QuIPI/data/clean/quipi_log10_tpm.pi")

In [ ]:
quipi_raw["archetype"] = quipi_raw["archetype"].fillna(value = "Unclassified")
quipi_log2["archetype"] = quipi_log2["archetype"].fillna(value = "Unclassified")
quipi_log10["archetype"] = quipi_log10["archetype"].fillna(value = "Unclassified")

In [ ]:
quipi_raw.to_pickle("quipi_raw_tpm.pi")
quipi_log2.to_pickle("quipi_log2_tpm.pi")
quipi_log10.to_pickle("quipi_log10_tpm.pi")

In [ ]:
inspect.signature(px.imshow)

In [ ]:
quipi_raw.shape

In [ ]:
df["indication"].unique()

In [ ]:
genes = ["indication","NCAM1", "DLK1", "IGF2","TAF1B","TOMM20P1", "ARL9"]

In [ ]:
df_2 = df[genes][df[genes]["indication"] == "ADR"].drop("indication",axis=1)
df_2

In [ ]:
#corr_df = df[["NCAM1", "DLK1", "IGF2","TAF1B","TOMM20P1", "ARL9"]].corr(method="pearson")
corr_df = df_2.corr(method="pearson")
tril = np.tril(corr_df, -1)
np.tril_indices(corr_df.shape[0], -1)
#corr_df[np.tril_indices(corr_df.shape[0], -1)] = np.nan
px.imshow(tril,color_continuous_scale = "RdBu",zmin=-1,zmax=1)

In [ ]:
corr_df

# Create a mask for the upper triangle
mask = np.triu(np.ones_like(corr_df, dtype=bool))

# Apply the mask to the correlation matrix
corr_lower_tri = corr_df.mask(mask)

corr_lower_tri = corr_lower_tri.dropna(axis=0, how = "all")
corr_lower_tri = corr_lower_tri.dropna(axis=1, how = "all")

corr_lower_tri


In [ ]:
fig = px.imshow(corr_lower_tri,
         zmin = -1, zmax =1,
         color_continuous_scale = "RdBu",
         text_auto=True)
fig.update_layout(template='simple_white')
fig.update_xaxes(showgrid=False, showline=False)
fig.update_yaxes(showgrid=False, showline=False)
#fig.update_yaxes(showgrid=False, zeroline=True)

fig.show()

In [ ]:
# Create a Plotly heatmap
heatmap = go.Heatmap(
    z=corr_lower_tri.values,
    x=corr_df.index,
    y=corr_df.columns,
    colorscale='RdBu',
    zmin=-1,
    zmax=1,
    text=corr_lower_tri.round(2).values,
    hoverinfo='text',
    showscale=True
)

fig = go.Figure(data=[heatmap])
fig.show()


In [ ]:
px.imshow(df[["NCAM1", "DLK1", "IGF2","TAF1B","TOMM20P1", "ARL9"]].corr(method="pearson"),
          origin="upper",text_auto=True,
         zmin=-1, zmax=1)

In [ ]:
df = pd.DataFrame.from_dict({"first" : [1, 0] , "second" : [0,1]})
df

In [ ]:
px.imshow(df.corr())

# Plotting Tests

## UMAP Example

In [ ]:
import inspect
inspect.signature(px.scatter)

In [ ]:
test = master_df.groupby("Indication")
px.box(test)

In [ ]:
px.box(master_df, x=["Indication","Compartment"], y="LYVE1")

In [ ]:
fig = master_df[["LYVE1"]].groupby("Indication").plot.box()
fig.show()

# Iso-Seq Stuff

In [ ]:
# Keep colors consistent across isoform classifications
color_map = {
    'antisense': "#FF6692",
    'full-splice_match': '#636EFA',
    'fusion': "#FFA15A",
    'genic': "#19D3F3",
    'incomplete-splice_match': "#AB63FA",
    'intergenic': "#B6E880",
    'moreJunctions' : "#FECB52",
    'novel_in_catalog': '#00CC96',
    'novel_not_in_catalog': "#EF553B"}

### IGF2

In [ ]:
# Read in GREP TSV
igf2_all_data = pd.read_csv("/Users/hwismer/Documents/p1_lr/data/B01_seurat/b01_igf2_grep.tsv", sep = "\t")
igf2_all_data

In [ ]:
igf2_df = igf2_all_data[igf2_all_data["gene"] == "IGF2"]
igf2_df

In [ ]:
# PLOT OF FORMS OF IGF2 Genes

igf2_gene_counts = pd.DataFrame(igf2_all_data["gene"].value_counts()).reset_index()
fig = px.pie(igf2_gene_counts, values = "count", names = "gene")
fig.update_layout(title_text= "IGF2 {Normal} Forms: " + str(len(igf2_all_data)) + " counts", title_x=.9)
fig.update_layout(autosize=False, width=500, height=500)
fig.update_layout(legend=dict(y=0.25,x=1))
#fig.write_image("/Users/hwismer/Documents/b01_igf2_genes.png", scale=4)

In [ ]:
# PLOT OF ISOFORM CLASSIFICATIONS OF IGF2

igf2_classes = pd.DataFrame(igf2_df["category"].value_counts()).reset_index()

fig = px.pie(igf2_classes, 
             values = "count", 
             names="category",
             color = "category",
             color_discrete_map = color_map)

fig.update_layout(title_text= "IGF2 {Normal} Isoforms: " + str(len(igf2_df)) + " counts", title_x=.45,title_y = .9)
fig.update_layout(autosize=False, width=600, height=500)
#fig.update_traces(textposition='inside', textinfo='value+percent')
fig.update_layout(legend=dict(y=0.5,x=1))
#fig.write_image("/Users/hwismer/Documents/b01_igf2_isoforms.png", scale=4)

In [ ]:
# PLOTS OF ISOFORM LENGTHS FOR IGF2

fig = px.histogram(igf2_df, x = "length", color = "category", color_discrete_map = color_map)
fig.update_layout(title_text= "IGF2 Isoform Lengths", title_x=0.5)
fig.update_layout(autosize=False, width=1000, height=500)
fig.update_layout(legend=dict(y=0.5,x=1))
#fig.write_image("/Users/hwismer/Documents/b01_igf2_lengths.png", scale=4)

## NCAM1

In [ ]:
# Read in NCAM grep tsv
ncam_data = pd.read_csv("/Users/hwismer/Documents/p1_lr/data/b01_seurat/b01_ncam_grep.tsv", sep = "\t")
ncam_data

In [ ]:
# Plots ISOFORM Breakdown

ncam_df = pd.DataFrame(ncam_data["category"].value_counts()).reset_index()

fig = px.pie(ncam_df, 
             values = "count", 
             names = "category",
             color = "category",
             color_discrete_map = color_map)

fig.update_layout(title_text= "NCAM1 {Normal} Isoforms: " + str(len(ncam_data)) + " counts", title_x=0.5, title_y = .8)
fig.update_layout(autosize=False, width=500, height=500)
fig.update_traces(textposition='inside', textinfo='value+percent')
fig.update_layout(legend=dict(y=.5,x=1))
fig.write_image("/Users/hwismer/Documents/b01_ncam1_isoforms.png", scale = 4)

In [ ]:
fig = px.histogram(ncam_data, x = "length", color = "category", color_discrete_map = color_map)
fig.update_layout(title_text= "NCAM1 Isoform Lengths", title_x=0.5)
fig.update_layout(autosize=False, width=1000, height=500)
fig.update_layout(legend=dict(y=0.5,x=1))
fig.write_image("/Users/hwismer/Documents/b01_ncam1_lengths.png", scale=4)

In [ ]:
a01_b01_df = pd.read_csv("/Users/hwismer/Documents/p1_lr/a01_b01_metadata.csv")
a01_b01_df = a01_b01_df.rename(columns = {"Unnamed: 0" : "cb"})
a01_b01_df

In [ ]:
sobj_df = pd.read_csv("/Users/hwismer/sobj_metadata.csv")
a01_df = sobj_df[sobj_df["orig.ident"] == "A01.Tumor"]
b01_df = sobj_df[sobj_df["orig.ident"] == "B01.Normal"]

a01_df.index = a01_df.index.str[:16]
b01_df.index = b01_df.index.str[:16]

a01_df["BC"] = a01_df.index
b01_df["BC"] = b01_df.index

In [ ]:
a01_isoforms_all = pd.read_csv("/Users/hwismer/Documents/p1_lr/data/A01_seurat/A01_collapsed.annotated.info.csv", sep = "\t")
b01_isoforms_all = pd.read_csv("/Users/hwismer/Documents/p1_lr/data/B01_seurat/B01_collapsed.annotated.info.csv", sep = "\t")

In [ ]:
a01_isoforms = a01_isoforms_all[a01_isoforms_all["BC"].isin(a01_df["BC"])]
b01_isoforms = b01_isoforms_all[b01_isoforms_all["BC"].isin(b01_df["BC"])]

In [ ]:
gene_set = {"DLK1", "NCAM1", "IGF2", "PLAG1", "LY6H", "MDK", "NTRK3"}

In [ ]:
a01_gene_set = a01_isoforms[a01_isoforms["gene"].isin(gene_set)]
b01_gene_set = b01_isoforms[b01_isoforms["gene"].isin(gene_set)]

#a01_bc_to_annot = a01_df["Annot"].to_dict()
#a01_gene_set_annots = [a01_bc_to_annot[bc] for bc in a01_gene_set["BC"]]

In [ ]:
a01_bc_to_annot = a01_df["Annot"].to_dict()
a01_gene_set_annots = [a01_bc_to_annot[bc] for bc in a01_gene_set["BC"]]
a01_gene_set["Annot"] = a01_gene_set_annots

b01_bc_to_annot = b01_df["Annot"].to_dict()
b01_gene_set_annots = [b01_bc_to_annot[bc] for bc in b01_gene_set["BC"]]
b01_gene_set["Annot"] = b01_gene_set_annots

#b01_gene_set = b01_isoforms[b01_isoforms["gene"].isin(gene_set)]

In [ ]:
color_map = {
    'antisense': "#FF6692",
    'full-splice_match': '#636EFA',
    'fusion': "#FFA15A",
    'genic': "#19D3F3",
    'incomplete-splice_match': "#AB63FA",
    'intergenic': "#B6E880",
    'moreJunctions' : "#FECB52",
    'novel_in_catalog': '#00CC96',
    'novel_not_in_catalog': "#EF553B"}

for gene in gene_set:
    
    gene_df = b01_gene_set[b01_gene_set["gene"] == gene]

    dfs = gene_df.groupby("Annot")["category"].value_counts().reset_index().groupby("Annot").sum()

    
    counts = gene_df.groupby("Annot")["category"].value_counts().reset_index()
    counts['% Composition'] = 100 * counts['count'] / counts.groupby('Annot')['count'].transform('sum')
    #ncam_df.groupby("Annot")["category"].value_counts()).reset_index()
    fig = px.bar(counts, x = "Annot", y = "% Composition", color = "category",
                 color_discrete_map = color_map,
                 hover_data="count",
                 title=gene)
    #fig.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False)
    
    fig.add_trace(go.Scatter(
        x=dfs.index,
        y=[100 for x in range(len(dfs))],
        text=dfs['count'],
        mode='text',
        textposition="top center",
        textfont=dict(
            size=10,
        ),
        showlegend=False
    ))
    
    fig.update_yaxes(range=[0,110])
    fig.update_layout(template='simple_white')
    fig.update_layout(title_text= gene, title_x=0.45,xaxis_title="")
    #fig.update_layout(autosize=True)#, width=600, height=400)
    fig.show()
    fig.write_image("/Users/hwismer/Desktop/{gene}_props.png".format(gene=gene),scale=8)